# 02. Huấn luyện mô hình PMIS

Notebook này thực hiện huấn luyện các mô hình ML/NLP cho hệ thống gợi ý thiết bị PMIS

## Mục lục
- A. Chuẩn bị dữ liệu
- B. Data augmentation
- C. Xây dựng mô hình
- D. Phát hiện lỗi/thiếu
- E. Huấn luyện & đánh giá
- F. Xuất mô hình

In [1]:
# Import thư viện
import pandas as pd
import numpy as np
import os
import sys
import pickle
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, top_k_accuracy_score
)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# NLP libraries
try:
    from sentence_transformers import SentenceTransformer
    HAS_SENTENCE_TRANSFORMERS = True
except Exception as e:
    HAS_SENTENCE_TRANSFORMERS = False
    print(f"⚠️ sentence-transformers không khả dụng: {type(e).__name__}")

# Thêm đường dẫn project
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(PROJECT_ROOT)

# Đường dẫn
DATA_DIR = os.path.join(PROJECT_ROOT, 'data')
MODEL_DIR = os.path.join(PROJECT_ROOT, 'models')
CONFIG_DIR = os.path.join(PROJECT_ROOT, 'config')
LOG_DIR = os.path.join(PROJECT_ROOT, 'logs')

# Tạo thư mục nếu chưa có
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(CONFIG_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

# File paths
INPUT_FILE = os.path.join(DATA_DIR, 'devicesPMISMayCat_cleaned.csv')

print(f"Project Root: {PROJECT_ROOT}")
print(f"Input File: {INPUT_FILE}")
print(f"Model Directory: {MODEL_DIR}")

2026-02-03 15:14:56.084863: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-03 15:14:56.113700: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-03 15:14:56.761720: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


⚠️ sentence-transformers không khả dụng: ValueError
Project Root: /home/aispcit/Documents/QuangLV/PMIS v 13
Input File: /home/aispcit/Documents/QuangLV/PMIS v 13/data/devicesPMISMayCat_cleaned.csv
Model Directory: /home/aispcit/Documents/QuangLV/PMIS v 13/models


## A. Chuẩn bị dữ liệu

In [2]:
# A.1. Đọc dữ liệu đã làm sạch
print("=" * 60)
print("A.1. ĐỌC DỮ LIỆU ĐÃ LÀM SẠCH")
print("=" * 60)

# Kiểm tra file tồn tại
if not os.path.exists(INPUT_FILE):
    # Nếu chưa có file cleaned, đọc file gốc
    INPUT_FILE = os.path.join(DATA_DIR, 'devicesPMISMayCat.csv')
    print(f"⚠️ File cleaned chưa tồn tại, đọc file gốc: {INPUT_FILE}")

df = pd.read_csv(INPUT_FILE, delimiter=';', encoding='utf-8')
print(f"Đã đọc {len(df):,} dòng dữ liệu")
print(f"Số cột: {len(df.columns)}")

A.1. ĐỌC DỮ LIỆU ĐÃ LÀM SẠCH
Đã đọc 1,696 dòng dữ liệu
Số cột: 32


In [3]:
# Định nghĩa các quy tắc chuẩn hóa
NORMALIZATION_RULES = {
    'PHA': 'EVN.PHA_3P',
    'KIEU_MC': 'TBI_CT_MC_KIEU_MC_01',
    'KIEU_DAPHQ': 'TBI_TT_MC_KIEU_DAPHQ.00001',
    'KIEU_CD': 'TBI_CT_MC_CC_CD.00001',
    'U_TT': 'TBI_CT_MC_U_TT_02',
}
FORBIDDEN_NATIONALFACT = 'TB040.00023'

In [4]:
# A.2. Áp dụng lại rules chuẩn hóa để đảm bảo 100%
print("=" * 60)
print("A.2. KIỂM TRA VÀ ÁP DỤNG QUY TẮC CHUẨN HÓA")
print("=" * 60)

# Thay thế 'NULL' string thành NaN
df = df.replace('NULL', np.nan)

# Kiểm tra các quy tắc
for col, expected_value in NORMALIZATION_RULES.items():
    if col in df.columns:
        non_standard = df[col].notna() & (df[col] != expected_value)
        print(f"{col}: {non_standard.sum()} giá trị không chuẩn (expected: {expected_value})")

# Kiểm tra NATIONALFACT
if 'NATIONALFACT' in df.columns:
    forbidden = (df['NATIONALFACT'] == FORBIDDEN_NATIONALFACT).sum()
    print(f"NATIONALFACT = {FORBIDDEN_NATIONALFACT}: {forbidden} bản ghi vi phạm")

A.2. KIỂM TRA VÀ ÁP DỤNG QUY TẮC CHUẨN HÓA
PHA: 0 giá trị không chuẩn (expected: EVN.PHA_3P)
KIEU_MC: 6 giá trị không chuẩn (expected: TBI_CT_MC_KIEU_MC_01)
KIEU_DAPHQ: 11 giá trị không chuẩn (expected: TBI_TT_MC_KIEU_DAPHQ.00001)
KIEU_CD: 273 giá trị không chuẩn (expected: TBI_CT_MC_CC_CD.00001)
U_TT: 1494 giá trị không chuẩn (expected: TBI_CT_MC_U_TT_02)
NATIONALFACT = TB040.00023: 0 bản ghi vi phạm


In [5]:
# A.3. Split train/val/test (70/15/15)
print("=" * 60)
print("A.3. CHIA TẬP DỮ LIỆU")
print("=" * 60)

# Định nghĩa các cột features và target
# Target: Gợi ý CATEGORYID hoặc các thuộc tính thiết bị
TARGET_COLS = ['LOAI', 'P_MANUFACTURERID']

# Features: Các cột không phải target và không phải _DESC
# Base features - se duoc dieu chinh tuy theo target
BASE_FEATURE_COLS = [
    'DATEMANUFACTURE', 'NATIONALFACT', 'OWNER',
    'U_TT', 'KIEU_DAPHQ', 'I_DM', 'U_DM', 'KIEU_CD',
    'TG_CATNM', 'PHA', 'KIEU_MC', 'KNCDNMDM', 'CT_DC'
]
FEATURE_COLS = BASE_FEATURE_COLS.copy()

# Text features (cho NLP)
# Text features - loai bo cac _DESC lien quan den target
TEXT_COLS = ['ASSETDESC', 'FIELDDESC', 'OWNER_DESC', 'KIEU_MC_DESC', 'KIEU_DAPHQ_DESC']

# Lọc các cột tồn tại
FEATURE_COLS = [col for col in FEATURE_COLS if col in df.columns]
TEXT_COLS = [col for col in TEXT_COLS if col in df.columns]

print(f"Feature columns: {len(FEATURE_COLS)}")
print(f"Text columns: {len(TEXT_COLS)}")

# Chia dữ liệu
# Loai bo cac dong co target la NaN
df_clean = df.dropna(subset=TARGET_COLS)
print(f"\nSo dong sau khi loai bo NaN trong target: {len(df_clean):,}")

# Hien thi phan bo cua cac target
for target in TARGET_COLS:
    print(f"\nPhan bo {target} (top 10):")
    dist = df_clean[target].value_counts().head(10)
    for val, count in dist.items():
        print(f"  {val}: {count:,} ({count/len(df_clean)*100:.1f}%)")

train_df, temp_df = train_test_split(df_clean, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(f"\nPhân chia dữ liệu:")
print(f"  Train: {len(train_df):,} ({len(train_df)/len(df)*100:.1f}%)")
print(f"  Validation: {len(val_df):,} ({len(val_df)/len(df)*100:.1f}%)")
print(f"  Test: {len(test_df):,} ({len(test_df)/len(df)*100:.1f}%)")

A.3. CHIA TẬP DỮ LIỆU
Feature columns: 13
Text columns: 5

So dong sau khi loai bo NaN trong target: 1,696

Phan bo LOAI (top 10):
  TBI_CT_MC_KIEU_MC.99020: 414 (24.4%)
  TBI_CT_MC_KIEU_MC.99019: 357 (21.0%)
  TBI_CT_MC_KIEU_MC.99010: 323 (19.0%)
  TBI_CT_MC_KIEU_MC.00057: 216 (12.7%)
  TBI_CT_MC_KIEU_MC.99001: 81 (4.8%)
  TBI_CT_MC_KIEU_MC.00071: 58 (3.4%)
  TBI_CT_MC_KIEU_MC.99006: 30 (1.8%)
  TBI_CT_MC_KIEU_MC.99018: 25 (1.5%)
  TBI_CT_MC_KIEU_MC.99005: 21 (1.2%)
  TBI_CT_MC_KIEU_MC.99017: 20 (1.2%)

Phan bo P_MANUFACTURERID (top 10):
  HSX.00311: 425 (25.1%)
  HSX.00046: 366 (21.6%)
  HSX.00035: 248 (14.6%)
  HSX.00505: 233 (13.7%)
  HSX.00051: 121 (7.1%)
  HSX.00183: 99 (5.8%)
  HSX.00092: 71 (4.2%)
  HSX.00508: 31 (1.8%)
  HSX.00535: 25 (1.5%)
  HSX.00473: 21 (1.2%)

Phân chia dữ liệu:
  Train: 1,187 (70.0%)
  Validation: 254 (15.0%)
  Test: 255 (15.0%)


## B.3. Xử lý Class Imbalance

> ⚠️ **LƯU Ý:** Các cell B.3.1 đến B.3.3 định nghĩa các hàm. Cell **B.3.4** cần được chạy **SAU** khi đã chạy các cells từ **Section C** (để có `X_train`, `y_train_dict`, `target_encoders`).

Áp dụng các kỹ thuật để cải thiện accuracy:
1. **SMOTE** - Synthetic Minority Over-sampling Technique
2. **Class Weights** - Cân bằng trọng số cho các class nhỏ
3. **Minimum Sample Threshold** - Loại bỏ hoặc gộp classes quá nhỏ

In [6]:
# B.3.1. Cài đặt và Import thư viện xử lý Class Imbalance
print("=" * 60)
print("B.3.1. THIẾT LẬP XỬ LÝ CLASS IMBALANCE")
print("=" * 60)

# Import SMOTE từ imbalanced-learn
try:
    from imblearn.over_sampling import SMOTE, ADASYN
    from imblearn.combine import SMOTETomek, SMOTEENN
    from imblearn.under_sampling import RandomUnderSampler
    HAS_IMBLEARN = True
    print("✓ imbalanced-learn đã được import thành công")
except ImportError:
    HAS_IMBLEARN = False
    print("⚠️ imbalanced-learn chưa cài đặt")
    print("   Cài đặt: pip install imbalanced-learn")

from collections import Counter

# Cấu hình
MIN_SAMPLES_PER_CLASS = 5  # Class có ít hơn sẽ được xử lý đặc biệt
SMOTE_K_NEIGHBORS = 3      # Số neighbors cho SMOTE (phải < min samples)

B.3.1. THIẾT LẬP XỬ LÝ CLASS IMBALANCE
✓ imbalanced-learn đã được import thành công


In [7]:
# B.3.2. Định nghĩa hàm Phân tích Class Imbalance
print("=" * 60)
print("B.3.2. ĐỊNH NGHĨA HÀM PHÂN TÍCH CLASS IMBALANCE")
print("=" * 60)

def analyze_class_imbalance(y, target_name, encoder=None):
    """Phân tích chi tiết class imbalance"""
    class_counts = Counter(y)
    n_classes = len(class_counts)
    total_samples = len(y)
    
    # Sắp xếp theo số lượng
    sorted_counts = sorted(class_counts.items(), key=lambda x: x[1], reverse=True)
    
    # Thống kê
    counts_list = list(class_counts.values())
    max_count = max(counts_list)
    min_count = min(counts_list)
    imbalance_ratio = max_count / min_count if min_count > 0 else float('inf')
    
    print(f"\n{target_name}:")
    print(f"  Tổng samples: {total_samples:,}")
    print(f"  Số classes: {n_classes}")
    print(f"  Max samples/class: {max_count}")
    print(f"  Min samples/class: {min_count}")
    print(f"  Imbalance ratio: {imbalance_ratio:.1f}:1")
    
    # Phân loại classes
    tiny_classes = [(c, cnt) for c, cnt in sorted_counts if cnt < MIN_SAMPLES_PER_CLASS]
    small_classes = [(c, cnt) for c, cnt in sorted_counts if MIN_SAMPLES_PER_CLASS <= cnt < 10]
    medium_classes = [(c, cnt) for c, cnt in sorted_counts if 10 <= cnt < 50]
    large_classes = [(c, cnt) for c, cnt in sorted_counts if cnt >= 50]
    
    print(f"\n  Phân loại classes:")
    print(f"    - Tiny (< {MIN_SAMPLES_PER_CLASS} samples): {len(tiny_classes)} classes")
    print(f"    - Small (5-9 samples): {len(small_classes)} classes")
    print(f"    - Medium (10-49 samples): {len(medium_classes)} classes")
    print(f"    - Large (≥ 50 samples): {len(large_classes)} classes")
    
    if tiny_classes:
        print(f"\n  ⚠️ Tiny classes (cần xử lý đặc biệt):")
        for cls_id, cnt in tiny_classes[:10]:
            if encoder is not None:
                try:
                    cls_name = encoder.inverse_transform([cls_id])[0]
                except:
                    cls_name = cls_id
            else:
                cls_name = cls_id
            print(f"      {cls_name}: {cnt} samples")
        if len(tiny_classes) > 10:
            print(f"      ... và {len(tiny_classes) - 10} classes khác")
    
    return {
        'total': total_samples,
        'n_classes': n_classes,
        'imbalance_ratio': imbalance_ratio,
        'tiny_classes': tiny_classes,
        'small_classes': small_classes,
        'class_counts': class_counts
    }

print("✓ Hàm analyze_class_imbalance đã được định nghĩa")
print("  (Sẽ được gọi sau khi y_train_dict được tạo)")

B.3.2. ĐỊNH NGHĨA HÀM PHÂN TÍCH CLASS IMBALANCE
✓ Hàm analyze_class_imbalance đã được định nghĩa
  (Sẽ được gọi sau khi y_train_dict được tạo)


In [8]:
# B.3.3. Hàm xử lý Class Imbalance
print("=" * 60)
print("B.3.3. ĐỊNH NGHĨA HÀM XỬ LÝ CLASS IMBALANCE")
print("=" * 60)

def compute_class_weights(y):
    """Tính class weights để cân bằng"""
    from sklearn.utils.class_weight import compute_class_weight
    classes = np.unique(y)
    weights = compute_class_weight('balanced', classes=classes, y=y)
    return dict(zip(classes, weights))

def filter_minority_classes(X, y, min_samples=MIN_SAMPLES_PER_CLASS):
    """Loại bỏ các samples thuộc class có quá ít mẫu"""
    class_counts = Counter(y)
    valid_classes = [c for c, cnt in class_counts.items() if cnt >= min_samples]
    
    mask = np.isin(y, valid_classes)
    X_filtered = X[mask]
    y_filtered = y[mask]
    
    removed_samples = len(y) - len(y_filtered)
    removed_classes = len(class_counts) - len(valid_classes)
    
    print(f"  Loại bỏ {removed_samples} samples từ {removed_classes} classes có < {min_samples} mẫu")
    
    return X_filtered, y_filtered, valid_classes

def apply_smote(X, y, k_neighbors=SMOTE_K_NEIGHBORS, strategy='auto'):
    """Áp dụng SMOTE để oversample minority classes"""
    if not HAS_IMBLEARN:
        print("  ⚠️ SMOTE không khả dụng (cần cài imbalanced-learn)")
        return X, y
    
    # Kiểm tra số samples tối thiểu cho SMOTE
    class_counts = Counter(y)
    min_samples = min(class_counts.values())
    
    if min_samples <= k_neighbors:
        print(f"  ⚠️ Min class có {min_samples} samples < k_neighbors={k_neighbors}")
        print(f"  → Điều chỉnh k_neighbors = {min_samples - 1}")
        k_neighbors = max(1, min_samples - 1)
    
    try:
        smote = SMOTE(k_neighbors=k_neighbors, random_state=42)
        X_resampled, y_resampled = smote.fit_resample(X, y)
        
        print(f"  SMOTE: {len(y):,} → {len(y_resampled):,} samples")
        
        # Hiển thị phân bố mới
        new_counts = Counter(y_resampled)
        print(f"  Phân bố sau SMOTE:")
        print(f"    Min samples/class: {min(new_counts.values())}")
        print(f"    Max samples/class: {max(new_counts.values())}")
        
        return X_resampled, y_resampled
    except Exception as e:
        print(f"  ⚠️ SMOTE failed: {e}")
        return X, y

def prepare_balanced_data(X_train, y_train, target_name, use_smote=True, use_filter=True):
    """Pipeline xử lý class imbalance hoàn chỉnh"""
    print(f"\n  Xử lý class imbalance cho {target_name}:")
    
    X_balanced = X_train.copy()
    y_balanced = y_train.copy()
    valid_classes = np.unique(y_train)
    
    # Bước 1: Lọc classes quá nhỏ (nếu cần)
    if use_filter:
        X_balanced, y_balanced, valid_classes = filter_minority_classes(
            X_balanced, y_balanced, min_samples=MIN_SAMPLES_PER_CLASS
        )
    
    # Bước 2: Áp dụng SMOTE (nếu có)
    if use_smote and HAS_IMBLEARN and len(np.unique(y_balanced)) > 1:
        X_balanced, y_balanced = apply_smote(X_balanced, y_balanced)
    
    # Bước 3: Tính class weights
    class_weights = compute_class_weights(y_balanced)
    
    return X_balanced, y_balanced, class_weights, valid_classes

print("✓ Các hàm xử lý class imbalance đã được định nghĩa")

B.3.3. ĐỊNH NGHĨA HÀM XỬ LÝ CLASS IMBALANCE
✓ Các hàm xử lý class imbalance đã được định nghĩa


In [9]:
# B.3.5. Cập nhật Models với Class Weights
print("=" * 60)
print("B.3.5. CẬP NHẬT MODELS VỚI CLASS WEIGHTS")
print("=" * 60)

def create_models_with_weights(class_weights=None):
    """Tạo models với class weights để xử lý imbalance"""
    
    # Model 1: Random Forest với class weights
    rf_model = RandomForestClassifier(
        n_estimators=200,           # Tăng số cây
        max_depth=15,               # Tăng độ sâu
        min_samples_split=5,        # Tránh overfit
        min_samples_leaf=2,
        class_weight=class_weights, # SỬ DỤNG CLASS WEIGHTS
        random_state=42,
        n_jobs=-1
    )
    
    # Model 2: Gradient Boosting (không hỗ trợ class_weight trực tiếp)
    gb_model = GradientBoostingClassifier(
        n_estimators=150,           # Tăng số iterations
        max_depth=6,
        learning_rate=0.1,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42
    )
    
    # Model 3: KNN (sử dụng weights='distance')
    knn_model = KNeighborsClassifier(
        n_neighbors=5,
        weights='distance',
        metric='cosine'
    )
    
    return {
        'RandomForest': rf_model,
        'GradientBoosting': gb_model,
        'KNN': knn_model
    }

print("✓ Hàm create_models_with_weights đã được định nghĩa")

B.3.5. CẬP NHẬT MODELS VỚI CLASS WEIGHTS
✓ Hàm create_models_with_weights đã được định nghĩa


## B. Data Augmentation

In [10]:
# B.1. Kiểm tra phân bố dữ liệu
print("=" * 60)
print("B.1. KIỂM TRA PHÂN BỐ DỮ LIỆU")
print("=" * 60)

# Phân bố theo CATEGORYID
for target in TARGET_COLS:
    if target in df.columns:
        print(f"\nPhan bo {target} (top 10):")
        dist = df[target].value_counts().head(10)
        for val, count in dist.items():
            print(f"  {val}: {count:,} ({count/len(df)*100:.1f}%)")

# Phân bố theo nhà sản xuất
if 'P_MANUFACTURERID' in df.columns:
    print(f"\nPhân bố nhà sản xuất (top 10):")
    mfg_dist = df['P_MANUFACTURERID'].value_counts().head(10)
    for mfg, count in mfg_dist.items():
        print(f"  {mfg}: {count:,}")

B.1. KIỂM TRA PHÂN BỐ DỮ LIỆU

Phan bo LOAI (top 10):
  TBI_CT_MC_KIEU_MC.99020: 414 (24.4%)
  TBI_CT_MC_KIEU_MC.99019: 357 (21.0%)
  TBI_CT_MC_KIEU_MC.99010: 323 (19.0%)
  TBI_CT_MC_KIEU_MC.00057: 216 (12.7%)
  TBI_CT_MC_KIEU_MC.99001: 81 (4.8%)
  TBI_CT_MC_KIEU_MC.00071: 58 (3.4%)
  TBI_CT_MC_KIEU_MC.99006: 30 (1.8%)
  TBI_CT_MC_KIEU_MC.99018: 25 (1.5%)
  TBI_CT_MC_KIEU_MC.99005: 21 (1.2%)
  TBI_CT_MC_KIEU_MC.99017: 20 (1.2%)

Phan bo P_MANUFACTURERID (top 10):
  HSX.00311: 425 (25.1%)
  HSX.00046: 366 (21.6%)
  HSX.00035: 248 (14.6%)
  HSX.00505: 233 (13.7%)
  HSX.00051: 121 (7.1%)
  HSX.00183: 99 (5.8%)
  HSX.00092: 71 (4.2%)
  HSX.00508: 31 (1.8%)
  HSX.00535: 25 (1.5%)
  HSX.00473: 21 (1.2%)

Phân bố nhà sản xuất (top 10):
  HSX.00311: 425
  HSX.00046: 366
  HSX.00035: 248
  HSX.00505: 233
  HSX.00051: 121
  HSX.00183: 99
  HSX.00092: 71
  HSX.00508: 31
  HSX.00535: 25
  HSX.00473: 21


In [11]:
# B.2. Synthetic data generation (nếu cần)
print("=" * 60)
print("B.2. DATA AUGMENTATION")
print("=" * 60)

def augment_text_data(df, text_col, num_augments=2):
    """
    Tạo synthetic samples bằng cách biến đổi text
    - Thêm/bỏ khoảng trắng
    - Đổi chữ hoa/thường
    - Thêm typo nhẹ
    """
    augmented_rows = []
    
    for idx, row in df.iterrows():
        if pd.notna(row[text_col]):
            text = str(row[text_col])
            
            # Variation 1: Upper case
            new_row = row.copy()
            new_row[text_col] = text.upper()
            augmented_rows.append(new_row)
            
            # Variation 2: Lower case
            new_row = row.copy()
            new_row[text_col] = text.lower()
            augmented_rows.append(new_row)
    
    return pd.DataFrame(augmented_rows)

# Kiểm tra nếu cần augmentation
MIN_SAMPLES_PER_CLASS = 10
need_augmentation = False

# Kiem tra cho tung target
for target in TARGET_COLS:
    if target in train_df.columns:
        class_counts = train_df[target].value_counts()
        small_classes = class_counts[class_counts < MIN_SAMPLES_PER_CLASS]
        if len(small_classes) > 0:
            need_augmentation = True
            print(f"\n{target} - Cac class co it hon {MIN_SAMPLES_PER_CLASS} samples:")
            for cls, count in small_classes.items():
                print(f"  {cls}: {count}")

if need_augmentation and 'ASSETDESC' in train_df.columns:
    print("\n⚠️ Cần data augmentation cho các class nhỏ")
    # Uncomment để thực hiện augmentation
    # augmented_df = augment_text_data(train_df[train_df[TARGET_COL].isin(small_classes.index)], 'ASSETDESC')
    # train_df = pd.concat([train_df, augmented_df], ignore_index=True)
    # print(f"Sau augmentation: {len(train_df):,} samples")
else:
    print("✓ Không cần data augmentation")

B.2. DATA AUGMENTATION

LOAI - Cac class co it hon 10 samples:
  PB-500158: 8
  TBI_CT_MC_KIEU_MC.99003: 7
  TBI_CT_MC_KIEU_MC.99004: 7
  TBI_CT_MC_KIEU_MC.000102: 6
  TBI_CT_MC_KIEU_MC.00069: 6
  TBI_CT_MC_KIEU_MC.00055: 5
  TBI_CT_MC_KIEU_MC.99034: 4
  TBI_CT_MC_KIEU_MC.99035: 3
  TBI_CT_MC_KIEU_MC.99002: 3
  TBI_CT_MC_KIEU_MC.99032: 2
  TBI_CT_MC_KIEU_MC.99015: 2
  TBI_CT_MC_KIEU_MC.99029: 2
  TBI_CT_MC_KIEU_MC.99031: 2
  TBI_CT_MC_KIEU_MC.99026: 2
  TBI_CT_MC_KIEU_MC.99030: 2
  TBI_CT_MC_KIEU_MC.99033: 1
  TBI_CT_MC_KIEU_MC.99027: 1
  TBI_CT_MC_KIEU_MC.99036: 1
  TBI_CT_MC_KIEU_MC.99012: 1

P_MANUFACTURERID - Cac class co it hon 10 samples:
  HSX.00417: 9
  HSX.00513: 5
  HSX.00544: 4
  PB-100056: 3
  HSX.T.662: 3
  HSX.T.700: 2
  HSX.T.704: 2
  HSX.00203: 2
  HSX.00529: 1
  HSX.00299: 1
  HSX.00290: 1
  PB-100109: 1
  HSX.00507: 1
  PB-100016: 1
  HSX.T.687: 1
  HSX.00184: 1

⚠️ Cần data augmentation cho các class nhỏ


## C. Xây dựng mô hình

In [12]:
# C.1. Chuẩn bị encoders
print("=" * 60)
print("C.1. CHUẨN BỊ ENCODERS")
print("=" * 60)

# Label encoders cho các cột categorical
label_encoders = {}
for col in FEATURE_COLS:
    if df[col].dtype == 'object':
        le = LabelEncoder()
        # Fit trên toàn bộ dữ liệu để bao gồm tất cả categories
        all_values = df[col].fillna('_MISSING_').astype(str).unique()
        le.fit(all_values)
        label_encoders[col] = le
        print(f"{col}: {len(le.classes_)} classes")

# TF-IDF cho text columns
tfidf_vectorizers = {}
for col in TEXT_COLS:
    tfidf = TfidfVectorizer(max_features=100, ngram_range=(1, 2))
    text_data = df[col].fillna('').astype(str)
    tfidf.fit(text_data)
    tfidf_vectorizers[col] = tfidf
    print(f"TF-IDF {col}: {len(tfidf.get_feature_names_out())} features")

C.1. CHUẨN BỊ ENCODERS
NATIONALFACT: 11 classes
OWNER: 2 classes
U_TT: 5 classes
KIEU_DAPHQ: 2 classes
I_DM: 5 classes
U_DM: 5 classes
KIEU_CD: 2 classes
TG_CATNM: 2 classes
PHA: 1 classes
KIEU_MC: 3 classes
KNCDNMDM: 5 classes
CT_DC: 1 classes
TF-IDF ASSETDESC: 100 features
TF-IDF FIELDDESC: 23 features
TF-IDF OWNER_DESC: 6 features
TF-IDF KIEU_MC_DESC: 3 features
TF-IDF KIEU_DAPHQ_DESC: 4 features


In [13]:
# C.2. Tạo feature matrix
print("=" * 60)
print("C.2. TẠO FEATURE MATRIX")
print("=" * 60)

def create_features(df, label_encoders, tfidf_vectorizers):
    """Tạo feature matrix từ DataFrame"""
    features_list = []
    feature_names = []
    
    # Categorical features
    for col, le in label_encoders.items():
        values = df[col].fillna('_MISSING_').astype(str)
        # Handle unseen labels
        encoded = []
        for v in values:
            if v in le.classes_:
                encoded.append(le.transform([v])[0])
            else:
                encoded.append(-1)  # Unknown
        features_list.append(np.array(encoded).reshape(-1, 1))
        feature_names.append(col)
    
    # Text features (TF-IDF)
    for col, tfidf in tfidf_vectorizers.items():
        text_data = df[col].fillna('').astype(str)
        tfidf_features = tfidf.transform(text_data).toarray()
        features_list.append(tfidf_features)
        feature_names.extend([f"{col}_tfidf_{i}" for i in range(tfidf_features.shape[1])])
    
    # Numeric features
    if 'DATEMANUFACTURE' in df.columns:
        date_feat = df['DATEMANUFACTURE'].fillna(df['DATEMANUFACTURE'].median()).values.reshape(-1, 1)
        features_list.append(date_feat)
        feature_names.append('DATEMANUFACTURE')
    
    # Concatenate all features
    X = np.hstack(features_list)
    
    return X, feature_names

# Tạo features cho train/val/test
X_train, feature_names = create_features(train_df, label_encoders, tfidf_vectorizers)
X_val, _ = create_features(val_df, label_encoders, tfidf_vectorizers)
X_test, _ = create_features(test_df, label_encoders, tfidf_vectorizers)

print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Total features: {len(feature_names)}")

C.2. TẠO FEATURE MATRIX
X_train shape: (1187, 149)
X_val shape: (254, 149)
X_test shape: (255, 149)
Total features: 149


In [14]:
# C.3. Chuẩn bị target variable
print("=" * 60)
print("C.3. CHUẨN BỊ TARGET VARIABLE")
print("=" * 60)

# Encode target
# Encode targets - tao encoder cho moi target
target_encoders = {}
y_train_dict = {}
y_val_dict = {}
y_test_dict = {}

for target in TARGET_COLS:
    le = LabelEncoder()
    # Fit tren TOAN BO du lieu (df_clean) de bao gom tat ca cac gia tri
    le.fit(df_clean[target].fillna('_UNKNOWN_'))
    y_train_dict[target] = le.transform(train_df[target].fillna('_UNKNOWN_'))
    y_val_dict[target] = le.transform(val_df[target].fillna('_UNKNOWN_'))
    y_test_dict[target] = le.transform(test_df[target].fillna('_UNKNOWN_'))
    target_encoders[target] = le

for target, le in target_encoders.items():
    print(f"\n{target}:")
    print(f"  Number of classes: {len(le.classes_)}")
    if len(le.classes_) <= 10:
        print(f"  Classes: {list(le.classes_)}")
    else:
        print(f"  Classes (top 10): {list(le.classes_[:10])}...")

C.3. CHUẨN BỊ TARGET VARIABLE

LOAI:
  Number of classes: 34
  Classes (top 10): ['PB-500017', 'PB-500158', 'TBI_CT_MC_KIEU_MC.000102', 'TBI_CT_MC_KIEU_MC.00055', 'TBI_CT_MC_KIEU_MC.00057', 'TBI_CT_MC_KIEU_MC.00069', 'TBI_CT_MC_KIEU_MC.00071', 'TBI_CT_MC_KIEU_MC.99001', 'TBI_CT_MC_KIEU_MC.99002', 'TBI_CT_MC_KIEU_MC.99003']...

P_MANUFACTURERID:
  Number of classes: 28
  Classes (top 10): ['HSX.00029', 'HSX.00035', 'HSX.00046', 'HSX.00051', 'HSX.00092', 'HSX.00183', 'HSX.00184', 'HSX.00203', 'HSX.00290', 'HSX.00299']...


In [15]:
# C.4. Xây dựng ML Model (gợi ý thiết bị)
print("=" * 60)
print("C.4. XÂY DỰNG ML MODEL")
print("=" * 60)

# Model 1: Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)

# Model 2: Gradient Boosting
gb_model = GradientBoostingClassifier(
    n_estimators=100,
    max_depth=5,
    random_state=42
)

# Model 3: KNN (cho similarity search)
knn_model = KNeighborsClassifier(
    n_neighbors=5,
    metric='cosine'
)

models = {
    'RandomForest': rf_model,
    'GradientBoosting': gb_model,
    'KNN': knn_model
}

print(f"Đã định nghĩa {len(models)} mô hình:")
for name in models.keys():
    print(f"  - {name}")

C.4. XÂY DỰNG ML MODEL
Đã định nghĩa 3 mô hình:
  - RandomForest
  - GradientBoosting
  - KNN


In [16]:
# C.5. NLP Model (chuẩn hóa text)
print("=" * 60)
print("C.5. NLP MODEL (CHUẨN HÓA TEXT)")
print("=" * 60)

class TextNormalizer:
    """Chuẩn hóa text input từ OCR"""
    
    def __init__(self):
        self.replacements = {
            # Các từ viết tắt phổ biến
            'MC': 'Máy cắt',
            'MBA': 'Máy biến áp',
            'TI': 'Máy biến dòng',
            'TU': 'Máy biến điện áp',
            'DCL': 'Dao cách ly',
        }
        
    def normalize(self, text):
        """Chuẩn hóa một chuỗi text"""
        if pd.isna(text):
            return ''
        
        text = str(text).strip()
        # Remove extra whitespace
        text = ' '.join(text.split())
        # Upper case for consistency
        text = text.upper()
        
        return text
    
    def normalize_batch(self, texts):
        """Chuẩn hóa một batch texts"""
        return [self.normalize(t) for t in texts]

text_normalizer = TextNormalizer()
print("✓ TextNormalizer đã được khởi tạo")

# Test
test_texts = ['  mc 171  ', 'máy cắt 172', 'MC-132_DSO']
print(f"\nTest normalization:")
for t in test_texts:
    print(f"  '{t}' -> '{text_normalizer.normalize(t)}'")

C.5. NLP MODEL (CHUẨN HÓA TEXT)
✓ TextNormalizer đã được khởi tạo

Test normalization:
  '  mc 171  ' -> 'MC 171'
  'máy cắt 172' -> 'MÁY CẮT 172'
  'MC-132_DSO' -> 'MC-132_DSO'


In [17]:
# C.6. Sentence Embeddings (nếu có)
print("=" * 60)
print("C.6. SENTENCE EMBEDDINGS")
print("=" * 60)

if HAS_SENTENCE_TRANSFORMERS:
    # Load pre-trained model
    embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    print("✓ Loaded sentence-transformers model")
    
    # Tạo embeddings cho ASSETDESC
    if 'ASSETDESC' in df.columns:
        sample_texts = df['ASSETDESC'].head(5).fillna('').tolist()
        sample_embeddings = embedding_model.encode(sample_texts)
        print(f"Sample embedding shape: {sample_embeddings.shape}")
else:
    print("⚠️ sentence-transformers không khả dụng")
    print("Cài đặt: pip install sentence-transformers")

C.6. SENTENCE EMBEDDINGS
⚠️ sentence-transformers không khả dụng
Cài đặt: pip install sentence-transformers


## E.0 Multi-Target Training (LOAI va P_MANUFACTURERID)

Cell nay thay the cac buoc E.1, E.2, E.3 cu de train models cho nhieu targets.

In [18]:
# E.0. MULTI-TARGET TRAINING
print("=" * 60)
print("E.0. MULTI-TARGET TRAINING")
print("=" * 60)

import time
from sklearn.base import clone

# Luu ket qua cho moi target
all_results = {}

for target in TARGET_COLS:
    print(f"\n{'='*60}")
    print(f"TARGET: {target}")
    print(f"{'='*60}")
    
    # Lay y values cho target hien tai
    y_train = y_train_dict[target]
    y_val = y_val_dict[target]
    y_test = y_test_dict[target]
    target_encoder = target_encoders[target]
    
    n_classes = len(target_encoder.classes_)
    print(f"Number of classes: {n_classes}")
    
    # Train models
    target_models = {}
    target_times = {}
    
    for name, model_template in models.items():
        model = clone(model_template)
        print(f"\n  Training {name}...")
        start = time.time()
        model.fit(X_train, y_train)
        elapsed = time.time() - start
        target_models[name] = model
        target_times[name] = elapsed
        print(f"    Done in {elapsed:.2f}s")
    
    # Validation
    print(f"\n  VALIDATION RESULTS:")
    val_results = {}
    for name, model in target_models.items():
        y_pred = model.predict(X_val)
        acc = accuracy_score(y_val, y_pred)
        val_results[name] = acc
        print(f"    {name}: Accuracy = {acc:.4f}")
    
    # Chon best model
    best_name = max(val_results, key=lambda x: val_results[x])
    best_model = target_models[best_name]
    print(f"\n  Best model: {best_name}")
    
    # Test evaluation
    y_pred_test = best_model.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred_test)
    test_prec = precision_score(y_test, y_pred_test, average='weighted', zero_division=0)
    test_rec = recall_score(y_test, y_pred_test, average='weighted', zero_division=0)
    test_f1 = f1_score(y_test, y_pred_test, average='weighted', zero_division=0)
    
    print(f"\n  TEST RESULTS ({best_name}):")
    print(f"    Accuracy:  {test_acc:.4f}")
    print(f"    Precision: {test_prec:.4f}")
    print(f"    Recall:    {test_rec:.4f}")
    print(f"    F1-Score:  {test_f1:.4f}")
    
    # Top-K Accuracy (chi cho multi-class)
    if n_classes > 2 and hasattr(best_model, 'predict_proba'):
        y_proba = best_model.predict_proba(X_test)
        # So classes tu model (co the it hon encoder neu train set khong co tat ca classes)
        model_n_classes = y_proba.shape[1]
        # Chi tinh top-k neu model co du classes
        if model_n_classes >= 3:
            # Loc y_test chi giu cac samples co class trong model
            model_classes = best_model.classes_
            valid_mask = np.isin(y_test, model_classes)
            if valid_mask.sum() > 0:
                y_test_filtered = y_test[valid_mask]
                y_proba_filtered = y_proba[valid_mask]
                labels = model_classes
                for k in [1, 3, 5]:
                    if k <= model_n_classes:
                        try:
                            top_k = top_k_accuracy_score(y_test_filtered, y_proba_filtered, k=k, labels=labels)
                            print(f"    Top-{k} Accuracy: {top_k:.4f}")
                        except Exception as e:
                            print(f"    Top-{k} Accuracy: N/A ({type(e).__name__})")
    
    # Luu ket qua
    all_results[target] = {
        'best_model_name': best_name,
        'best_model': best_model,
        'all_models': target_models,
        'target_encoder': target_encoder,
        'metrics': {
            'accuracy': test_acc,
            'precision': test_prec,
            'recall': test_rec,
            'f1': test_f1
        }
    }

print(f"\n{'='*60}")
print("HOAN TAT MULTI-TARGET TRAINING")
print(f"{'='*60}")

E.0. MULTI-TARGET TRAINING

TARGET: LOAI
Number of classes: 34

  Training RandomForest...
    Done in 0.09s

  Training GradientBoosting...
    Done in 7.97s

  Training KNN...
    Done in 0.00s

  VALIDATION RESULTS:
    RandomForest: Accuracy = 0.6535
    GradientBoosting: Accuracy = 0.7165
    KNN: Accuracy = 0.6142

  Best model: GradientBoosting

  TEST RESULTS (GradientBoosting):
    Accuracy:  0.7490
    Precision: 0.7508
    Recall:    0.7490
    F1-Score:  0.7457
    Top-1 Accuracy: 0.7520
    Top-3 Accuracy: 0.9134
    Top-5 Accuracy: 0.9409

TARGET: P_MANUFACTURERID
Number of classes: 28

  Training RandomForest...
    Done in 0.10s

  Training GradientBoosting...
    Done in 6.47s

  Training KNN...
    Done in 0.00s

  VALIDATION RESULTS:
    RandomForest: Accuracy = 0.6811
    GradientBoosting: Accuracy = 0.7913
    KNN: Accuracy = 0.6417

  Best model: GradientBoosting

  TEST RESULTS (GradientBoosting):
    Accuracy:  0.8118
    Precision: 0.8307
    Recall:    0.8118


In [19]:
# F.0. LUU MULTI-TARGET MODELS
print("=" * 60)
print("F.0. LUU MULTI-TARGET MODELS")
print("=" * 60)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Luu models cho moi target
for target, result in all_results.items():
    print(f"\n{target}:")
    
    # Luu best model
    model_path = os.path.join(MODEL_DIR, f'{target.lower()}_classifier_{timestamp}.pkl')
    with open(model_path, 'wb') as f:
        pickle.dump(result['best_model'], f)
    print(f"  Model: {model_path}")
    
    # Luu target encoder
    enc_path = os.path.join(MODEL_DIR, f'{target.lower()}_encoder_{timestamp}.pkl')
    with open(enc_path, 'wb') as f:
        pickle.dump(result['target_encoder'], f)
    print(f"  Encoder: {enc_path}")

# Luu label encoders va TF-IDF
encoders_path = os.path.join(MODEL_DIR, f'feature_encoders_{timestamp}.pkl')
with open(encoders_path, 'wb') as f:
    pickle.dump({
        'label_encoders': label_encoders,
        'tfidf_vectorizers': tfidf_vectorizers
    }, f)
print(f"\nFeature encoders: {encoders_path}")

# Luu config
config = {
    'timestamp': timestamp,
    'targets': TARGET_COLS,
    'feature_columns': FEATURE_COLS,
    'text_columns': TEXT_COLS,
    'results': {}
}

for target, result in all_results.items():
    config['results'][target] = {
        'best_model': result['best_model_name'],
        'n_classes': len(result['target_encoder'].classes_),
        'metrics': result['metrics']
    }

config_path = os.path.join(CONFIG_DIR, f'multi_target_config_{timestamp}.json')
with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)
print(f"\nConfig: {config_path}")

print("\nDONE!")

F.0. LUU MULTI-TARGET MODELS

LOAI:
  Model: /home/aispcit/Documents/QuangLV/PMIS v 13/models/loai_classifier_20260203_151512.pkl
  Encoder: /home/aispcit/Documents/QuangLV/PMIS v 13/models/loai_encoder_20260203_151512.pkl

P_MANUFACTURERID:
  Model: /home/aispcit/Documents/QuangLV/PMIS v 13/models/p_manufacturerid_classifier_20260203_151512.pkl
  Encoder: /home/aispcit/Documents/QuangLV/PMIS v 13/models/p_manufacturerid_encoder_20260203_151512.pkl

Feature encoders: /home/aispcit/Documents/QuangLV/PMIS v 13/models/feature_encoders_20260203_151512.pkl

Config: /home/aispcit/Documents/QuangLV/PMIS v 13/config/multi_target_config_20260203_151512.json

DONE!


In [20]:
# TONG KET
print("\n" + "=" * 60)
print("TONG KET MULTI-TARGET CLASSIFICATION")
print("=" * 60)

print(f"\nTargets: {TARGET_COLS}")
print(f"Training samples: {len(train_df):,}")
print(f"Test samples: {len(test_df):,}")

print(f"\nKET QUA:")
for target, result in all_results.items():
    m = result['metrics']
    print(f"\n  {target}:")
    print(f"    Best Model: {result['best_model_name']}")
    print(f"    Classes: {len(result['target_encoder'].classes_)}")
    print(f"    Accuracy: {m['accuracy']:.4f}")
    print(f"    F1-Score: {m['f1']:.4f}")


TONG KET MULTI-TARGET CLASSIFICATION

Targets: ['LOAI', 'P_MANUFACTURERID']
Training samples: 1,187
Test samples: 255

KET QUA:

  LOAI:
    Best Model: GradientBoosting
    Classes: 34
    Accuracy: 0.7490
    F1-Score: 0.7457

  P_MANUFACTURERID:
    Best Model: GradientBoosting
    Classes: 28
    Accuracy: 0.8118
    F1-Score: 0.8106


## D. Phát hiện lỗi/thiếu

In [21]:
# D.1. Rule-based detection
print("=" * 60)
print("D.1. RULE-BASED DETECTION")
print("=" * 60)

class RuleBasedDetector:
    """Phát hiện lỗi/thiếu dựa trên quy tắc"""
    
    def __init__(self, rules, forbidden_values):
        self.rules = rules
        self.forbidden_values = forbidden_values
    
    def detect(self, df):
        """Phát hiện các bản ghi có lỗi"""
        errors = []
        
        for idx, row in df.iterrows():
            row_errors = []
            
            # Check required fields
            for col, expected in self.rules.items():
                if col in row.index:
                    if pd.notna(row[col]) and row[col] != expected:
                        row_errors.append({
                            'column': col,
                            'type': 'non_standard_value',
                            'current': row[col],
                            'expected': expected
                        })
            
            # Check forbidden values
            for col, forbidden in self.forbidden_values.items():
                if col in row.index:
                    if row[col] == forbidden:
                        row_errors.append({
                            'column': col,
                            'type': 'forbidden_value',
                            'current': row[col]
                        })
            
            # Check missing critical fields
            critical_fields = ['ASSETID', 'ASSETDESC', 'CATEGORYID']
            for col in critical_fields:
                if col in row.index and pd.isna(row[col]):
                    row_errors.append({
                        'column': col,
                        'type': 'missing_critical',
                        'current': None
                    })
            
            if row_errors:
                errors.append({
                    'index': idx,
                    'assetid': row.get('ASSETID', 'N/A'),
                    'errors': row_errors
                })
        
        return errors

# Khởi tạo detector
rule_detector = RuleBasedDetector(
    rules=NORMALIZATION_RULES,
    forbidden_values={'NATIONALFACT': FORBIDDEN_NATIONALFACT}
)

# Chạy detection trên test set
detected_errors = rule_detector.detect(test_df.head(100))
print(f"Số bản ghi có lỗi (sample 100): {len(detected_errors)}")

if detected_errors:
    print(f"\nVí dụ lỗi phát hiện:")
    for err in detected_errors[:3]:
        print(f"  ASSETID: {err['assetid']}")
        for e in err['errors'][:2]:
            print(f"    - {e['column']}: {e['type']}")

D.1. RULE-BASED DETECTION
Số bản ghi có lỗi (sample 100): 92

Ví dụ lỗi phát hiện:
  ASSETID: PB-0110D00-MC17716798
    - KIEU_CD: non_standard_value
    - U_TT: non_standard_value
  ASSETID: PB-0110D00-MC15722842
    - U_TT: non_standard_value
  ASSETID: PB-0110D00-MC12647793
    - U_TT: non_standard_value


In [22]:
# D.2. ML-based anomaly detection
print("=" * 60)
print("D.2. ML-BASED ANOMALY DETECTION")
print("=" * 60)

from sklearn.ensemble import IsolationForest

# Sử dụng Isolation Forest
iso_forest = IsolationForest(
    n_estimators=100,
    contamination=0.05,  # 5% dữ liệu được coi là anomaly
    random_state=42
)

# Fit và predict
iso_forest.fit(X_train)
anomaly_scores = iso_forest.decision_function(X_test)
anomaly_labels = iso_forest.predict(X_test)

# Thống kê
n_anomalies = (anomaly_labels == -1).sum()
print(f"Số anomalies phát hiện: {n_anomalies} ({n_anomalies/len(X_test)*100:.1f}%)")

D.2. ML-BASED ANOMALY DETECTION
Số anomalies phát hiện: 14 (5.5%)


In [23]:
# D.3. Tự đề xuất giá trị thay thế
print("=" * 60)
print("D.3. ĐỀ XUẤT GIÁ TRỊ THAY THẾ")
print("=" * 60)

class ValueSuggester:
    """Đề xuất giá trị thay thế cho các trường thiếu/lỗi"""
    
    def __init__(self, df, rules):
        self.df = df
        self.rules = rules
        self.mode_values = {}
        
        # Tính mode cho mỗi cột
        for col in df.columns:
            if df[col].dtype == 'object':
                mode = df[col].mode()
                if len(mode) > 0:
                    self.mode_values[col] = mode[0]
    
    def suggest(self, column, current_value=None):
        """Đề xuất giá trị cho một cột"""
        suggestions = []
        
        # Rule-based suggestion
        if column in self.rules:
            suggestions.append({
                'value': self.rules[column],
                'source': 'rule',
                'confidence': 1.0
            })
        
        # Mode-based suggestion
        if column in self.mode_values:
            suggestions.append({
                'value': self.mode_values[column],
                'source': 'mode',
                'confidence': 0.8
            })
        
        return suggestions

value_suggester = ValueSuggester(train_df, NORMALIZATION_RULES)

# Test suggestions
print("Test đề xuất giá trị:")
for col in ['PHA', 'KIEU_MC', 'OWNER']:
    if col in df.columns:
        suggestions = value_suggester.suggest(col)
        print(f"\n{col}:")
        for s in suggestions:
            print(f"  - {s['value']} (source: {s['source']}, conf: {s['confidence']})")

D.3. ĐỀ XUẤT GIÁ TRỊ THAY THẾ
Test đề xuất giá trị:

PHA:
  - EVN.PHA_3P (source: rule, conf: 1.0)
  - EVN.PHA_3P (source: mode, conf: 0.8)

KIEU_MC:
  - TBI_CT_MC_KIEU_MC_01 (source: rule, conf: 1.0)
  - TBI_CT_MC_KIEU_MC_01 (source: mode, conf: 0.8)

OWNER:
  - TB0632 (source: mode, conf: 0.8)
